# Pitcher WAR/162 Projection – Multi-Year t → t+1 (Optimized for Pitchers)

This notebook predicts **next-year WAR per 162 games (WAR/162)** for pitchers from
**current-year features** using the multi-year dataset:

`data/Pitchers_2015-2025_byYear_retry.csv`

Design decisions are tuned for **pitchers**:

- Build a **year t → year t+1** panel for all seasons.
- Restrict to pitchers with **meaningful workloads** in both seasons to reduce noise
  (default: `IP_t ≥ 30` and `IP_{t+1} ≥ 30`).
- Drop features that are missing in >60% of rows (e.g., sparse Statcast / bat-tracking fields).
- Two feature configurations:
  - `skill_only`: excludes current-year WAR/WAR_per_162 (pure skill & context).
  - `with_prev_war`: includes current-year WAR and WAR_per_162 (realistic projection setup).
- Models:
  - **ElasticNetCV** (regularized linear regression).
  - **MLPRegressor** (simple NN for tabular data).
- Train on all transitions with `Season ≤ 2023` and hold out `Season == 2024` (predicting 2025).


In [ ]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (7, 7)


## 1. Load pitcher-by-year dataset and compute WAR_per_162

In [ ]:
data_path = 'data/Pitchers_2015-2025_byYear_retry.csv'
df = pd.read_csv(data_path)
print('Shape:', df.shape)
print('Columns (first 25):', list(df.columns)[:25])
print('\nSeason value counts:')
print(df['Season'].value_counts().sort_index())

df = df.copy()
df['WAR_per_162'] = (df['WAR'] / df['G'].replace(0, np.nan)) * 162
print('\nSample WAR_per_162 rows:')
print(df[['mlbID', 'Season', 'G', 'WAR', 'WAR_per_162']].head())


## 2. Build year t → year t+1 panel

We construct pairs where:
- Features come from season `t`.
- Target (`WAR_per_162_next`) comes from season `t+1` for the same pitcher.

In [ ]:
min_season, max_season = df['Season'].min(), df['Season'].max()
print('Seasons span:', min_season, 'to', max_season)

# t side: all but last season
df_t = df[df['Season'] < max_season].copy()

# t+1 target side
df_target = df[['mlbID', 'Season', 'WAR_per_162', 'IP']].copy()
df_target = df_target.rename(columns={
    'Season': 'Season_next',
    'WAR_per_162': 'WAR_per_162_next',
    'IP': 'IP_next',
})
df_target['Season'] = df_target['Season_next'] - 1

panel = df_t.merge(
    df_target[['mlbID', 'Season', 'Season_next', 'WAR_per_162_next', 'IP_next']],
    on=['mlbID', 'Season'],
    how='inner'
)

print('Panel shape (t -> t+1 pairs):', panel.shape)
print('\nPairs by t (Season):')
print(panel['Season'].value_counts().sort_index())
print('\nPairs by t+1 (Season_next):')
print(panel['Season_next'].value_counts().sort_index())

# Drop missing targets
panel = panel.dropna(subset=['WAR_per_162', 'WAR_per_162_next'])
print('\nPanel after dropping missing WAR_per_162:', panel.shape)


## 3. Filter to pitchers with meaningful workloads

Next-year WAR is extremely noisy for tiny samples and cups of coffee.
We restrict to pitchers who have **at least a minimum IP** in both t and t+1.

Default:
- `MIN_IP_T = 30`
- `MIN_IP_TP1 = 30`

You can relax or tighten these thresholds depending on your use case.

In [ ]:
MIN_IP_T = 30.0     # minimum IP in season t
MIN_IP_TP1 = 30.0   # minimum IP in season t+1

workload_mask = (panel['IP'] >= MIN_IP_T) & (panel['IP_next'] >= MIN_IP_TP1)
panel_w = panel[workload_mask].copy()

print('Panel (workload-filtered) shape:', panel_w.shape)
print('\nWorkload-filtered pairs by t (Season):')
print(panel_w['Season'].value_counts().sort_index())

# Correlation of WAR_per_162_t vs WAR_per_162_{t+1} after filter (sanity upper bound)
corr = panel_w[['WAR_per_162', 'WAR_per_162_next']].corr().iloc[0, 1]
print(f"Correlation WAR_per_162_t vs WAR_per_162_t+1 (filtered): {corr:.3f}")


## 4. Define numeric feature columns and drop highly missing features

We start from all numeric columns, excluding:
- Identifiers and year markers: `mlbID`, `Season`, `Season_next`.
- Target: `WAR_per_162_next`.

Then we drop any feature with >60% missing values in the workload-filtered panel
to avoid letting extremely sparse Statcast/bat-tracking features dominate.

In [ ]:
# Identify numeric columns
numeric_cols = panel_w.select_dtypes(include=[np.number]).columns.tolist()
exclude_base = {'mlbID', 'Season', 'Season_next', 'WAR_per_162_next'}
base_feature_cols = [c for c in numeric_cols if c not in exclude_base]
print('Initial numeric feature count (incl current-year WAR):', len(base_feature_cols))

# Drop features with >60% missingness
missing_frac = panel_w[base_feature_cols].isna().mean()
too_sparse = missing_frac[missing_frac > 0.60].index.tolist()
print('Features with >60% missingness:', len(too_sparse))

base_feature_cols = [c for c in base_feature_cols if c not in too_sparse]
print('Final feature count after dropping sparse columns:', len(base_feature_cols))
print('Example features:', base_feature_cols[:25])


## 5. Train/test split (multi-year)

- **Train**: all transitions with `Season ≤ 2023` (predicting up through 2024).
- **Test**: transitions with `Season == 2024` (predicting 2025).

In [ ]:
train_panel = panel_w[panel_w['Season'] <= 2023].copy()
test_panel = panel_w[panel_w['Season'] == 2024].copy()

print('Train panel shape:', train_panel.shape)
print('Test panel shape :', test_panel.shape)

y_train_full = train_panel['WAR_per_162_next'].values
y_test_full = test_panel['WAR_per_162_next'].values


## 6. Helper: run one experiment (feature_mode + models)

We support two feature modes:
- `skill_only`: drop current-year `WAR` and `WAR_per_162` from features.
- `with_prev_war`: keep them as features (captures persistence in true talent & usage).


In [ ]:
def run_experiment(train_panel, test_panel, base_feature_cols, feature_mode='skill_only', random_state=42):
    # Start from base features
    feature_cols = base_feature_cols.copy()

    if feature_mode == 'skill_only':
        feature_cols = [c for c in feature_cols if c not in {'WAR', 'WAR_per_162'}]
    elif feature_mode == 'with_prev_war':
        # Keep WAR and WAR_per_162 as features
        pass
    else:
        raise ValueError(f'Unknown feature_mode: {feature_mode}')

    # Drop any rows with missing target just in case
    train_subset = train_panel.dropna(subset=['WAR_per_162_next']).copy()
    test_subset = test_panel.dropna(subset=['WAR_per_162_next']).copy()

    X_train = train_subset[feature_cols].values
    y_train = train_subset['WAR_per_162_next'].values
    X_test = test_subset[feature_cols].values
    y_test = test_subset['WAR_per_162_next'].values

    # Impute + scale
    imputer = SimpleImputer(strategy='median')
    scaler = StandardScaler()

    X_train_imp = imputer.fit_transform(X_train)
    X_test_imp = imputer.transform(X_test)

    X_train_scaled = scaler.fit_transform(X_train_imp)
    X_test_scaled = scaler.transform(X_test_imp)

    # ElasticNetCV
    elastic_cv = ElasticNetCV(
        l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],
        alphas=np.logspace(-3, 2, 20),
        cv=5,
        random_state=random_state,
        n_jobs=-1
    )
    elastic_cv.fit(X_train_scaled, y_train)
    y_train_pred_lr = elastic_cv.predict(X_train_scaled)
    y_test_pred_lr = elastic_cv.predict(X_test_scaled)

    lr_train_rmse = mean_squared_error(y_train, y_train_pred_lr, squared=False)
    lr_test_rmse = mean_squared_error(y_test, y_test_pred_lr, squared=False)
    lr_train_r2 = r2_score(y_train, y_train_pred_lr)
    lr_test_r2 = r2_score(y_test, y_test_pred_lr)

    # Neural net
    mlp = MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation='relu',
        alpha=1e-4,
        learning_rate_init=1e-3,
        max_iter=500,
        early_stopping=True,
        random_state=random_state
    )
    mlp.fit(X_train_scaled, y_train)
    y_train_pred_nn = mlp.predict(X_train_scaled)
    y_test_pred_nn = mlp.predict(X_test_scaled)

    nn_train_rmse = mean_squared_error(y_train, y_train_pred_nn, squared=False)
    nn_test_rmse = mean_squared_error(y_test, y_test_pred_nn, squared=False)
    nn_train_r2 = r2_score(y_train, y_train_pred_nn)
    nn_test_r2 = r2_score(y_test, y_test_pred_nn)

    return {
        'feature_mode': feature_mode,
        'n_train': len(y_train),
        'n_test': len(y_test),
        'feature_count': len(feature_cols),
        'feature_names': feature_cols,
        'lr_train_rmse': lr_train_rmse,
        'lr_test_rmse': lr_test_rmse,
        'lr_train_r2': lr_train_r2,
        'lr_test_r2': lr_test_r2,
        'nn_train_rmse': nn_train_rmse,
        'nn_test_rmse': nn_test_rmse,
        'nn_train_r2': nn_train_r2,
        'nn_test_r2': nn_test_r2,
        'elastic_model': elastic_cv,
        'mlp_model': mlp,
        'y_test': y_test,
        'y_test_pred_lr': y_test_pred_lr,
        'y_test_pred_nn': y_test_pred_nn,
    }


## 7. Run experiments for `skill_only` and `with_prev_war`

We run both feature configurations and compare test RMSE / R².

In [ ]:
feature_modes = ['skill_only', 'with_prev_war']
all_results = []

for fm in feature_modes:
    print(f"\n=== Running experiment: feature_mode={fm} ===")
    res = run_experiment(train_panel, test_panel, base_feature_cols, feature_mode=fm, random_state=42)
    all_results.append(res)

summary_rows = []
for r in all_results:
    summary_rows.append({
        'feature_mode': r['feature_mode'],
        'n_train': r['n_train'],
        'n_test': r['n_test'],
        'feature_count': r['feature_count'],
        'lr_test_rmse': r['lr_test_rmse'],
        'lr_test_r2': r['lr_test_r2'],
        'nn_test_rmse': r['nn_test_rmse'],
        'nn_test_r2': r['nn_test_r2'],
    })

summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.sort_values(by='nn_test_r2', ascending=False)
summary_df


## 8. Inspect best configuration and its features

We pick the configuration with the highest NN test R² and print its feature names.

In [ ]:
best_idx = int(np.argmax([r['nn_test_r2'] for r in all_results]))
best_res = all_results[best_idx]

print('Best config:')
print(' feature_mode:', best_res['feature_mode'])
print(' nn_test_r2  :', best_res['nn_test_r2'])
print('\nFeatures used (best model):')
print(best_res['feature_names'])


## 9. Diagnostics: predicted vs actual WAR/162 (test set) for best NN


In [ ]:
y_test = best_res['y_test']
y_pred = best_res['y_test_pred_nn']

plt.figure()
plt.scatter(y_test, y_pred, alpha=0.7)
lims = [min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())]
plt.plot(lims, lims, linestyle='--')
plt.xlabel('Actual WAR/162 (t+1, test)')
plt.ylabel('Predicted WAR/162 (t+1, test)')
plt.title(f"Predicted vs Actual WAR/162 – NN | {best_res['feature_mode']}")
plt.grid(True, alpha=0.3)
plt.show()
